In [ ]:
# https://pypi.org/project/requests/

In [5]:
import requests
import math

In [189]:
def getCoordinates(adresse):
    adresse = adresse.replace(" ","+")
    requete = "https://api-adresse.data.gouv.fr/search/?q="+adresse
    response = requests.get(requete)
    json = response.json()
    lat = json["features"][0]["geometry"]["coordinates"][1]
    lon = json["features"][0]["geometry"]["coordinates"][0]
    return lat,lon

In [190]:
def convRadiusKM(radius_km):
    return math.pow(radius_km/111.1949,2)

# Main

In [192]:
adresse = input("Veuillez saisir une adresse comlète.ex: 10 rue de Paris Paris 75001 \n").replace(" ","+")
lat, lon = getCoordinates(adresse)
radius_km = int(input("Saisir la portée en km"))
radius_degree = convRadiusKM(radius_km)

Veuillez saisir une adresse comlète.ex: 10 rue de Paris Paris 75001 
  


IndexError: list index out of range

### Testing

In [193]:
adresse = "5 Rue de l'Horloge, 35000 Rennes"
lat, lon = getCoordinates(adresse)
radius_km = 10
radius_degree = convRadiusKM(radius_km)
phi = round(lat)
val1 = math.cos(phi)*math.sin(phi)* math.pi/180
print(val1)
val2 = math.cos(phi)*(math.cos(phi) - math.sin(phi)* math.pi/180 * (lat-2*phi))
print(val2)

0.00858342232004857
0.8208332568911848


### Query building

In [194]:
PREFIX = "PREFIX ns: <http://schema.org/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#>"

In [195]:
FILTER = "FILTER (("+str(lat)+"-?lat)*("+str(lat)+"-?lat) + ("+str(lon)+"-?long)*("+str(lon)+"-?long)*("+str(val2)+"-("+str(val1)+"*?lat)) < "+str(radius_degree)+")}"

In [196]:
requete = "SELECT ?ville ?name ?available_bikes ?available_bikes_stands ?bike_stands ?lat ?long ?place WHERE { " \
"?station rdf:type ns:station . ?station ns:name ?name . ?station ns:ville ?ville . ?station ns:available_bikes ?available_bikes . "  \
"?station ns:available_bikes_stands ?available_bikes_stands . ?station ns:long ?long . ?station ns:lat ?lat . ?station ns:place ?place . "

In [197]:
sparql_query = PREFIX + requete + FILTER

In [198]:
sparql_query

'PREFIX ns: <http://schema.org/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#>SELECT ?ville ?name ?available_bikes ?available_bikes_stands ?bike_stands ?lat ?long ?place WHERE { ?station rdf:type ns:station . ?station ns:name ?name . ?station ns:ville ?ville . ?station ns:available_bikes ?available_bikes . ?station ns:available_bikes_stands ?available_bikes_stands . ?station ns:long ?long . ?station ns:lat ?lat . ?station ns:place ?place . FILTER ((48.111353-?lat)*(48.111353-?lat) + (-1.680345-?long)*(-1.680345-?long)*(0.8208332568911848-(0.00858342232004857*?lat)) < 0.008087797384722423)}'

In [199]:
import urllib


resultat_json = response.json()

In [200]:
url = "http://localhost:3030/db/sparql?query="+sparql_query
response = requests.get(url)
response.json()

{'head': {'vars': ['ville',
   'name',
   'available_bikes',
   'available_bikes_stands',
   'bike_stands',
   'lat',
   'long',
   'place']},
 'results': {'bindings': []}}

In [201]:
url

'http://localhost:3030/db/sparql?query=PREFIX%20ns%3A%20%3Chttp%3A%2F%2Fschema.org%2F%3E%20PREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%20%20PREFIX%20rdfs%3A%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%20PREFIX%20owl%3A%20%3Chttp%3A%2F%2Fwww.owl-ontologies.com%2Funnamed.owl%23%3ESELECT%20%3Fville%20%3Fname%20%3Favailable_bikes%20%3Favailable_bikes_stands%20%3Fbike_stands%20%3Flat%20%3Flong%20%3Fplace%20WHERE%20%7B%20%3Fstation%20rdf%3Atype%20ns%3Astation%20.%20%3Fstation%20ns%3Aname%20%3Fname%20.%20%3Fstation%20ns%3Aville%20%3Fville%20.%20%3Fstation%20ns%3Aavailable_bikes%20%3Favailable_bikes%20.%20%3Fstation%20ns%3Aavailable_bikes_stands%20%3Favailable_bikes_stands%20.%20%3Fstation%20ns%3Along%20%3Flong%20.%20%3Fstation%20ns%3Alat%20%3Flat%20.%20%3Fstation%20ns%3Aplace%20%3Fplace%20.%20FILTER%20%28%2848.111353-%3Flat%29%2A%2848.111353-%3Flat%29%20%2B%20%28-1.680345-%3Flong%29%2A%28-1.680345-%3Flong%29%2A%280.8208332568911848-%280.00858342232

In [187]:
# https://pypi.org/project/sparql-client/

In [188]:
#cd /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages 

In [59]:
resultat_json["results"]

### Exemple Liste des stations de vélos 10 km autour de la mairie de Lyon

In [280]:
import pandas as pd

In [281]:
adresse = "19 Quai Tilsitt, 69002 Lyon"
radius_km = 2
resultat = requete(adresse,radius_km)
resultat

,name,place,ville,lat,lon
0,Croix Rousse / Perfetti,station,Lyon,45.77225384153049e0,4.819460862010004e0
1,Gambetta / Garibaldi,station,Lyon,45.75179013554747e0,4.852607966008752e0
2,Perrache Est,station,Lyon,45.74610307357447e0,4.825605393944585e0
3,Perrache / Carnot,station,Lyon,45.75009964332607e0,4.828051501355155e0
4,Tabareau,station,Lyon,45.77408067053439e0,4.825159712198364e0
...,...,...,...,...,...
169,Créqui / Voltaire,station,Lyon,45.75669834417519e0,4.848403485755715e0
170,Guillotière / Veyron,station,Lyon,45.7479327825068e0,4.856337181809767e0
171,Gorge de Loup,station,Lyon,45.76736547851909e0,4.805627699519277e0
172,Place Ennemond Fousseret,station,Lyon,45.76468150567123e0,4.828983326522516e0


In [279]:
def requete(adresse,radius):
    lat, lon = getCoordinates(adresse)
    radius_degree = convRadiusKM(radius_km)
    phi = round(lat)
    val1 = math.cos(phi)*math.sin(phi)* math.pi/180
    val2 = math.cos(phi)*(math.cos(phi) - math.sin(phi)* math.pi/180 * (lat-2*phi))
    
    # Lyon Velos
    PREFIX = "PREFIX ns: <http://schema.org/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>" \
    "  PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#> PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#>"
    
    FILTER = "FILTER (("+str(lat)+"-?lat)*("+str(lat)+"-?lat) + ("+str(lon)+"-?long)*("+str(lon)+"-?long)* " \
    "("+str(val2)+"-("+str(val1)+"*?lat)) < "+str(radius_degree)+")}"
    
    requete = "SELECT ?ville ?name ?available_bikes ?available_bikes_stands ?bike_stands ?lat ?long ?place WHERE { " \
    "?station rdf:type ns:station . ?station ns:name ?name . ?station ns:ville ?ville . ?station ns:available_bikes ?available_bikes . "  \
    "?station ns:available_bikes_stands ?available_bikes_stands . ?station ns:long ?long . ?station ns:lat ?lat . ?station ns:place ?place . "
    sparql_query = PREFIX + requete + FILTER
    #print(sparql_query)
    sparql_query = urllib.parse.quote(sparql_query, safe='')
    url = "http://localhost:3030/db/sparql?query="+sparql_query
    response = requests.get(url)
    json = response.json()
    json = json["results"]["bindings"]
    df = pd.DataFrame()
    
    for i in range(len(json)):
        name = json[i]["name"]["value"] 
        ville = json[i]["ville"]["value"]
        lat = json[i]["lat"]["value"]
        lon = json[i]["long"]["value"]
        place = json[i]["place"]["value"]
        list_to_append = [name] + [ville] + [lat] + [lon] + [place]
        line_labels = ["name"] + ["ville"] + ["lat"] + ["lon"] + ["place"]
        dic = {line_labels[i]: list_to_append[i] for i in range(len(list_to_append))}
        df = df.append(dic,ignore_index=True)
    
    df = df [["name","place","ville","lat","lon"]]
    return df

In [ ]:
PREFIX ns: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:<http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.owl-ontologies.com/unnamed.owl#>

SELECT ?ville ?lat ?long ?place
WHERE {
	 ?hospital rdf:type ns:hospital .
  	?hospital ns:ville ?ville .	
  	?hospital ns:long ?long .
  	?hospital ns:lat ?lat .
  	?hospital ns:place ?place .
  	FILTER(?ville="Lille") .
}